# Procesamiento de datos de gravedad

Importamos librerías que vamos a utilizar

In [ ]:
import pathlib
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
import pygmt
import pyvista as pv

import ensaio
import verde as vd
import boule as bl
import harmonica as hm

## Descargamos datos de gravedad

In [ ]:
raw_data = pd.read_csv(fname_gravity)
raw_data

In [ ]:
fig = pygmt.Figure()
gmt_projection = "M12c"

raw_region = vd.get_region((raw_data.longitude, raw_data.latitude))

pygmt.makecpt(cmap="gray")
fig.grdimage(
    "@earth_relief_01m",
    region=raw_region,
    projection=gmt_projection,
    shading="+a45+nt0.7",
    cmap=True,
)
fig.coast(
    water="#8fcae7",
    transparency=50,
)
pygmt.makecpt(
    cmap="viridis",
    series=[raw_data.gravity_mgal.min(), raw_data.gravity_mgal.max()],
)
fig.plot(
    x=raw_data.longitude,
    y=raw_data.latitude,
    fill=raw_data.gravity_mgal,
    cmap=True,
    style="c2p",
    projection=gmt_projection,
    frame="af",
)
fig.colorbar(frame='af+l"mGal"')
fig.show()

## Recortamos los datos al complejo igneo de Bushveld

In [ ]:
region = (25, 32, -27, -23)

In [ ]:
inside = vd.inside((raw_data.longitude, raw_data.latitude), region)
data = raw_data[inside]

In [ ]:
def plot_scatter(longitude, latitude, values, center=False, cmap="viridis", label=None):
    """Plot scatter points using PyGMT"""
    fig = pygmt.Figure()
    gmt_projection = "M12c"
    region = vd.get_region((longitude, latitude))
    
    pygmt.makecpt(cmap="gray")
    fig.grdimage(
        "@earth_relief_01m",
        region=region,
        projection=gmt_projection,
        shading="+a45+nt0.7",
        cmap=True,
    )
    if center:
        maxabs = vd.maxabs(values)
        series = [-maxabs, maxabs] 
    else:
        series=[np.nanmin(values), np.nanmax(values)]
        
    pygmt.makecpt(cmap=cmap, series=series)
    fig.plot(
        x=longitude,
        y=latitude,
        fill=values,
        cmap=True,
        style="c3p",
        projection=gmt_projection,
        frame="af",
    )
    frame = "af"
    if label is not None:
        frame += f'+l"{label}"'
    fig.colorbar(frame=frame)
    fig.show()

In [ ]:
plot_scatter(data.longitude, data.latitude, data.gravity_mgal, label="gravity [mgal]")

## Gravity disturbance (disturbio de gravedad)

In [ ]:
data = data.assign(disturbance=disturbance)
data

In [ ]:
plot_scatter(
    data.longitude,
    data.latitude,
    data.disturbance,
    cmap="polar",
    center=True,
    label="gravity disturbance [mgal]"
)

## Efecto topografico

Descargamos una grilla de topografia

In [ ]:
topo_fname = ensaio.fetch_earth_topography(version=1)

In [ ]:
topography = xr.load_dataarray(topo_fname)
topography

La recortamos alrededor de la zona de estudio

In [ ]:
region_pad = vd.pad_region(region, pad=2)
topography = topography.sel(longitude=slice(*region_pad[:2]), latitude=slice(*region_pad[2:]))
topography

In [ ]:
fig = pygmt.Figure()
gmt_projection = "M12c"
fig.grdimage(topography, projection=gmt_projection, cmap="etopo1", frame=True)
fig.colorbar(frame='af+l"topografia [m]"')

maxabs = vd.maxabs(data.disturbance)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.disturbance,
    cmap=True,
    style="c2p",
    projection=gmt_projection,
    frame="af",
)
fig.show()

Proyectamos la grilla a coordenadas planas

In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=data.latitude.mean())

In [ ]:
data = data.assign(easting=easting, northing=northing)
data

In [ ]:

topo_proj

Creamos un modelo de topografia con prismas rectangulares

In [ ]:
density = np.where(topo_proj < 0, 1040 - 2670, 2670)

prisms = hm.prism_layer(
    (topo_proj.easting, topo_proj.northing),
    reference=0,
    surface=topo_proj,
    properties={"density": density},
)
prisms

Ploteo 3D

In [ ]:
pv_grid = prisms.prism_layer.to_pyvista()
pv_grid["elevation"] = topo_proj.values.ravel()
pv_grid

In [ ]:
plotter = pv.Plotter(lighting="three_lights", window_size=(1080 * 16 // 8, 1080), notebook=True)
plotter.add_mesh(pv_grid, scalars="elevation", cmap="terrain", show_scalar_bar=False)
plotter.set_scale(zscale=75)  # exaggerate the vertical coordinate

plotter.camera_position = "xz"
plotter.camera.elevation = 20
plotter.camera.azimuth = 125
plotter.camera.zoom(1.7)

plotter.show_axes()
plotter.show()

Calculamos el efecto topografico en cada punto de observación

In [ ]:
coordinates = (data.easting, data.northing, data.height_sea_level_m)
topo_effect = 

In [ ]:
data = data.assign(bouguer=bouguer)
data

In [ ]:
plot_scatter(
    data.longitude,
    data.latitude,
    data.bouguer,
    cmap="polar",
    center=True,
    label="Bouguer disturbance [mgal]",
)

## Calcular la gravedad residual

Usamos fuentes equivalentes profundas para calcular el campo regional de gravedad.

In [ ]:
deep_sources = hm.EquivalentSources(depth=500e3, damping=1000)
deep_sources.fit(coordinates, data.bouguer)

In [ ]:
regional = deep_sources.predict(coordinates)

In [ ]:
plot_scatter(
    data.longitude,
    data.latitude,
    regional,
    cmap="viridis",
    label="regional [mgal]",
)

Calculamos residual como la diferencia entre Bouguer y la regional.

In [ ]:
data = data.assign(residual=residual)
data

In [ ]:
plot_scatter(
    data.longitude,
    data.latitude,
    data.residual,
    cmap="polar",
    center=True,
    label="Residual [mgal]",
)

## Grillado con fuentes equivalentes

Definimos fuentes equivalentes someras para grillar la gravedad residual

In [ ]:
eq_sources = hm.EquivalentSources(depth=10e3, damping=10)
eq_sources.fit(coordinates, data.residual)

In [ ]:
grid_coords = vd.grid_coordinates(region=region, spacing=0.01, extra_coords=2200)
grid = eq_sources.grid(
    coordinates=grid_coords,
    projection=projection,
    dims=("latitude", "longitude"),
    data_names="residual_gravity",
)
grid

In [ ]:
fig = pygmt.Figure()
gmt_projection = "M15c"

pygmt.makecpt(cmap="gray")
fig.grdimage(
    "@earth_relief_01m",
    region=region,
    projection=gmt_projection,
    shading="+a45+nt0.7",
    cmap=True,
)

maxabs = vd.maxabs(grid.residual_gravity)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs], no_bg=True)
fig.grdimage(
    grid.residual_gravity,
    shading="+a45+nt0.15",
    projection=gmt_projection,
    frame=True,
    nan_transparent=True,
    transparency=10,
)
fig.colorbar(frame='af+l"residual field [mGal]"')
fig.plot(
    x=data.longitude,
    y=data.latitude,
    style="c0.02c",
    fill="black",
)
fig.show()

In [ ]:
grid = vd.convexhull_mask(
    (data.longitude, data.latitude), grid=grid
)
grid

In [ ]:
fig = pygmt.Figure()
gmt_projection = "M15c"

pygmt.makecpt(cmap="gray")
fig.grdimage(
    "@earth_relief_01m",
    region=region,
    projection=gmt_projection,
    shading="+a45+nt0.7",
    cmap=True,
)

maxabs = vd.maxabs(grid.residual_gravity)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs], no_bg=True)
fig.grdimage(
    grid.residual_gravity,
    shading="+a45+nt0.15",
    projection=gmt_projection,
    frame=True,
    nan_transparent=True,
    transparency=10,
)
fig.colorbar(frame='af+l"residual field [mGal]"')
fig.plot(
    x=data.longitude,
    y=data.latitude,
    style="c0.02c",
    fill="black",
)
fig.show()

<img src="../images/bushveld_igneous_complex_geology.jpg" style="width: 50%">